**1. Adım: Kütüphaneleri ekliyoruz.**

In [3]:
import pandas as pd
import geopandas as gpd
import shapely.geometry
import mercantile
from tqdm import tqdm
import os
import tempfile
import fiona

In [2]:
!pip install mercantile

## 2. Adım: - İstediğiniz Alanı Tanımlama (AOI)

İstediğiniz bölgeyi (AOI) GeoJSON geometrisi olarak tanımlayınız, ardından sınırlayıcı alanı eklemek için "shapely" kütüphanesini kullanınız.

**Not**: GeoJSON için koordinat referans sistemi "EPSG:4326" olmalıdır, yani global enlem/boylam biçiminde olmalıdır. Bunun için https://geojson.io internet adresini kullanabilirsiniz.

In [4]:
# İndirmek istediğimiz alanı bu siteden işaretliyoruz --> https://geojson.io
aoi_geom = {
    "coordinates": [
        [
              [
              -76.68107587508712,
              39.32377012182852
            ],
            [
              -76.68107587508712,
              39.24439604939738
            ],
            [
              -76.55494608216614,
              39.24439604939738
            ],
            [
              -76.55494608216614,
              39.32377012182852
            ],
            [
              -76.68107587508712,
              39.32377012182852
            ]
        ]
    ],
    "type": "Polygon",
}
aoi_shape = shapely.geometry.shape(aoi_geom)
minx, miny, maxx, maxy = aoi_shape.bounds

#Buraya indirilecek dosyanın adını yazılıyor.
output_fn = "deneme1.geojson"

## 3. Adım: Hangi Tile'ların AOI ile kesiştiğini belirleme.

In [5]:
quad_keys = set()
for tile in list(mercantile.tiles(minx, miny, maxx, maxy, zooms=9)):
    quad_keys.add(int(mercantile.quadkey(tile)))
quad_keys = list(quad_keys)
print(f"The input area spans {len(quad_keys)} tiles: {quad_keys}")

The input area spans 2 tiles: [32010032, 32010033]


## 4.Adım - AOI ile kesişen her bir Tile için binaları indirme ve AOI ile kesme.

AOI'mizle kesişen her döşeme için tüm binaları indirir, ardından yalnızca AOI'nin içerdiği binaları tutar.

*Not*: AOI'nizin kaç parçayı kapladığına ve bu parçalarda kaç bina bulunduğuna bağlı olarak bu adım biraz zaman alabilir.

In [ ]:
df = pd.read_csv(
    "https://minedbuildings.blob.core.windows.net/global-buildings/dataset-links.csv"
)

idx = 0
combined_rows = []

with tempfile.TemporaryDirectory() as tmpdir:
    # AOI ile kesişen tile alanlarını indirme.
    tmp_fns = []
    for quad_key in tqdm(quad_keys):
        rows = df[df["QuadKey"] == quad_key]
        if rows.shape[0] == 1:
            url = rows.iloc[0]["Url"]

            df2 = pd.read_json(url, lines=True)
            df2["geometry"] = df2["geometry"].apply(shapely.geometry.shape)

            gdf = gpd.GeoDataFrame(df2, crs=4326)
            fn = os.path.join(tmpdir, f"{quad_key}.geojson")
            tmp_fns.append(fn)
            if not os.path.exists(fn):
                gdf.to_file(fn, driver="GeoJSON")
        elif rows.shape[0] > 1:
            raise ValueError(f"Multiple rows found for QuadKey: {quad_key}")
        else:
            raise ValueError(f"QuadKey not found in dataset: {quad_key}")

    # GeoJSON dosyaları birleştirme
    for fn in tmp_fns:
        with fiona.open(fn, "r") as f:
            for row in tqdm(f):
                row = dict(row)
                shape = shapely.geometry.shape(row["geometry"])

                if aoi_shape.contains(shape):
                    if "id" in row:
                        del row["id"]
                    row["properties"] = {"id": idx}
                    idx += 1
                    combined_rows.append(row)

 50%|█████     | 1/2 [05:27<05:27, 327.13s/it]

## Step 4 - Sonuçları Kaydetme ve indirme

In [ ]:
schema = {"geometry": "Polygon", "properties": {"id": "int"}}

with fiona.open(output_fn, "w", driver="GeoJSON", crs="EPSG:4326", schema=schema) as f:
    f.writerecords(combined_rows)